### LDA Topics clustering with Gensim

In [2]:
import pandas as pd
import gensim
from gensim import corpora, models

from modules.nlputils import StemFilterTokenizeProcessor

speeches_df = pd.read_csv('output/speeches/senate-speeches-201806.csv', index_col='CodigoPronunciamento')
speeches = speeches_df[['TextoIntegral']]
speeches = [s[0] for s in speeches.values[:]]

stopwords = set([line.strip() for line in open('stopwords.txt', 'r', encoding='utf-8')])
proc = StemFilterTokenizeProcessor(stopwords=stopwords, min_size=3, filter_regex='[a-z]', stem_language='portuguese', stem_complete=True, only_nouns=True)
tokens = proc.tokenize_documents(speeches)

dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(text) for text in tokens]
lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=4, id2word=dictionary, passes=100)
print(lda_model)
lda_model.print_topics(4)

LdaModel(num_terms=3763, num_topics=4, decay=0.5, chunksize=2000)


[(0,
  '0.010*"mundo" + 0.009*"violência" + 0.009*"gente" + 0.008*"resistência" + 0.007*"ministro" + 0.007*"tribunal" + 0.007*"mulheres" + 0.006*"polícia" + 0.006*"direito" + 0.006*"vida"'),
 (1,
  '0.012*"resistência" + 0.010*"gente" + 0.008*"mercado" + 0.008*"população" + 0.008*"energia" + 0.008*"preços" + 0.008*"educação" + 0.007*"preço" + 0.007*"bilhões" + 0.007*"petróleo"'),
 (2,
  '0.017*"rio" + 0.013*"mato" + 0.013*"grosso" + 0.012*"sul" + 0.010*"saúde" + 0.009*"região" + 0.009*"trabalho" + 0.007*"projeto" + 0.006*"cidade" + 0.006*"cidades"'),
 (3,
  '0.020*"projeto" + 0.015*"trabalhadores" + 0.011*"sessão" + 0.010*"câmara" + 0.009*"ordem" + 0.009*"preço" + 0.009*"direitos" + 0.008*"votação" + 0.008*"comissão" + 0.008*"deputados"')]

### LDA Topics clustering with Scikitlearn

In [3]:
import modules.scikitutils as scikitutils
from modules.nlputils import StemFilterTokenizeProcessor
import sys
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.datasets import make_multilabel_classification

speeches_df = pd.read_csv('output/speeches/senate-speeches-201806.csv', index_col='CodigoPronunciamento')
speeches = speeches_df[['TextoIntegral']]
speeches = [s[0] for s in speeches.values[:]]
print('All speeches 06/2018')
print('qtty=' + str(len(speeches)))

stopwords = set([line.strip() for line in open('stopwords.txt', 'r', encoding='utf-8')])
# stopwords = []
proc = StemFilterTokenizeProcessor(min_size=3, filter_regex='[a-z]', stem_language='portuguese', stem_complete=True, only_nouns=True)

tf_vectorizer = CountVectorizer(min_df=0.1, max_df=1.0, stop_words=stopwords, preprocessor=proc.process_text)
tf = tf_vectorizer.fit_transform(speeches)
tf_feature_names = tf_vectorizer.get_feature_names()

lda = LatentDirichletAllocation(n_topics=4, max_iter=200, learning_method='batch', random_state=0, n_jobs=2)
lda.fit(tf)

scikitutils.display_lda_topics(lda, tf_feature_names, 5)

All speeches 06/2018
qtty=466


/usr/local/lib/python3.5/dist-packages/sklearn/decomposition/online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Topic 0: trabalhadores preço mercado bilhões resistência 
Topic 1: rio trabalho sul mundo violência 
Topic 2: gente ministro ministério direito tribunal 
Topic 3: projeto câmara sessão ordem deputado 


### Clusterize with KMeans

In [4]:
import pandas as pd
from modules.nlputils import StemFilterTokenizeProcessor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import MiniBatchKMeans, KMeans

speeches_df = pd.read_csv('output/speeches/senate-speeches-201806.csv', index_col='CodigoPronunciamento')
speeches = speeches_df[['TextoIntegral']]
speeches = [s[0] for s in speeches.values[:]]

stopwords = set([line.strip() for line in open('stopwords.txt', 'r', encoding='utf-8')])
proc = StemFilterTokenizeProcessor(min_size=3, filter_regex='[a-z]', stem_language='portuguese', stem_complete=True, only_nouns=True)

vec = TfidfVectorizer(min_df=0.1, stop_words=stopwords, analyzer='word', ngram_range=(1, 2), preprocessor=proc.process_text)
X = vec.fit_transform(speeches)

number_of_clusters = 4
kmeans_model = KMeans(n_clusters=number_of_clusters)
speech_groups = kmeans_model.fit_predict(X)

order_centroids = kmeans_model.cluster_centers_.argsort()[:, ::-1]
terms = vec.get_feature_names()
cluster_labels = []
for i in range(number_of_clusters):
    top_words = [terms[ind] for ind in order_centroids[i, :5]]
    gl = "Cluster {}: {}".format(i, ' '.join(top_words))
    print(gl)
    cluster_labels.append(gl)

results = pd.DataFrame()
results['speeches'] = speeches
results['category'] = kmeans_model.labels_

for index, row in results[:5].iterrows():
    print('========')
    print(str(row['category']) + ' ' + cluster_labels[row['category']] + ' ' + row['speeches'])


Cluster 0: veto rio rio sul sul região
Cluster 1: preço petróleo gasolina setor nordeste
Cluster 2: resistência gente projeto mundo brasileiros
Cluster 3: ordem votação sessão deputado plenário
1 Cluster 1: preço petróleo gasolina setor nordeste      O SR. ROBERTO REQUIÃO  (Bloco Maioria/MDB - PR. Pronuncia o seguinte discurso. Sem revisão do orador.) – Senador Presidente, Senador Paim, vou aproveitar esta sexta-feira para repetir o mantra que eu venho repetindo com insistência sobre o que realmente está acontecendo no Brasil. Nós não estamos, Senador Paim, descolados de uma circunstância global. O que acontece no Brasil, acontece em outros países também.     O Papa Francisco, quando os governantes mais poderosos do mundo se reuniam em Davos, com os homens mais ricos da Terra, os concentradores do capital, dizia: "O capital é bom quando ele é investido, gera empregos, produtos, inova na tecnologia, mas o capital tem que servir à sociedade e não pode dominá-la." Na verdade, o que o Papa